<a href="https://colab.research.google.com/github/MocT117/Another-one-/blob/master/MADFINDER_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import re
from datetime import datetime

# ---------------------- #
# Configuración general  #
# ---------------------- #
INPUT_FOLDER = 'raw_data/'
OUTPUT_FOLDER = 'output/'

# ------------------------------- #
# Función: cargar archivos Excel  #
# ------------------------------- #
def cargar_archivos_excel(ruta_carpeta):
    archivos = [f for f in os.listdir(ruta_carpeta) if f.endswith('.xlsx')]
        df_total = pd.DataFrame()
            for archivo in archivos:
                    ruta_completa = os.path.join(ruta_carpeta, archivo)
                            try:
                                        df = pd.read_excel(ruta_completa, engine='openpyxl')
                                                    df_total = pd.concat([df_total, df], ignore_index=True)
                                                            except Exception as e:
                                                                        print(f"Error al leer {archivo}: {e}")
                                                                            return df_total

                                                                            # --------------------------------------------- #
                                                                            # Función: limpiar y crear la columna clave Key #
                                                                            # --------------------------------------------- #
                                                                            def crear_columna_key(df):
                                                                                df = df.copy()
                                                                                    # Asegura tipo string y elimina ceros iniciales en 'Sales order no.'
                                                                                        df['Sales order no.'] = df['Sales order no.'].astype(str).str.lstrip('0')
                                                                                            df['Material number'] = df['Material number'].astype(str)
                                                                                                df['Key'] = df['Sales order no.'] + '_' + df['Material number']
                                                                                                    return df

                                                                                                    # ---------------------------------------------------------- #
                                                                                                    # Función: extraer fechas MAD desde el texto (Regex robusto) #
                                                                                                    # ---------------------------------------------------------- #
                                                                                                    def extraer_fechas_mad(texto):
                                                                                                        """
                                                                                                            Extrae fechas asociadas a 'MAD' o 'NEW MAD' desde un texto y las normaliza a formato YYYY-MM-DD.
                                                                                                                """
                                                                                                                    if not isinstance(texto, str):
                                                                                                                            return []
                                                                                                                                patron = r'(?:MAD|NEW MAD)[:\s]*([0-9]{1,2}[./][0-9]{1,2}(?:[./][0-9]{2,4})?)'
                                                                                                                                    matches = re.findall(patron, texto, flags=re.IGNORECASE)
                                                                                                                                        fechas_limpias = []
                                                                                                                                            for fecha in matches:
                                                                                                                                                    try:
                                                                                                                                                                fecha = fecha.replace('/', '.')
                                                                                                                                                                            partes = fecha.split('.')
                                                                                                                                                                                        if len(partes) == 2:
                                                                                                                                                                                                        dia, mes = partes
                                                                                                                                                                                                                        año = '2025'
                                                                                                                                                                                                                                    elif len(partes) == 3:
                                                                                                                                                                                                                                                    dia, mes, año = partes
                                                                                                                                                                                                                                                                    if len(año) == 2:
                                                                                                                                                                                                                                                                                        año = '20' + año
                                                                                                                                                                                                                                                                                                    else:
                                                                                                                                                                                                                                                                                                                    continue
                                                                                                                                                                                                                                                                                                                                fecha_formateada = datetime(int(año), int(mes), int(dia)).strftime('%Y-%m-%d')
                                                                                                                                                                                                                                                                                                                                            fechas_limpias.append(fecha_formateada)
                                                                                                                                                                                                                                                                                                                                                    except Exception as e:
                                                                                                                                                                                                                                                                                                                                                                print(f"⚠️ Error al convertir '{fecha}' → {e}")
                                                                                                                                                                                                                                                                                                                                                                            continue
                                                                                                                                                                                                                                                                                                                                                                                return fechas_limpias

                                                                                                                                                                                                                                                                                                                                                                                # --------------------------------------------------------- #
                                                                                                                                                                                                                                                                                                                                                                                # Función: obtener la fecha MAD final para cada fila        #
                                                                                                                                                                                                                                                                                                                                                                                # --------------------------------------------------------- #
                                                                                                                                                                                                                                                                                                                                                                                def obtener_ultima_fecha_mad(row):
                                                                                                                                                                                                                                                                                                                                                                                    """
                                                                                                                                                                                                                                                                                                                                                                                        Retorna la última fecha MAD (más reciente) de la fila, usando primero 'Text sales order' y si no, 'Material staging'.
                                                                                                                                                                                                                                                                                                                                                                                            """
                                                                                                                                                                                                                                                                                                                                                                                                # 1. Si hay texto, buscamos fechas MAD
                                                                                                                                                                                                                                                                                                                                                                                                    fechas_mad = extraer_fechas_mad(row.get('Text sales order', ''))
                                                                                                                                                                                                                                                                                                                                                                                                        if fechas_mad:
                                                                                                                                                                                                                                                                                                                                                                                                                return max(fechas_mad)
                                                                                                                                                                                                                                                                                                                                                                                                                    else:
                                                                                                                                                                                                                                                                                                                                                                                                                            # 2. Si no hay texto, usamos Material staging
                                                                                                                                                                                                                                                                                                                                                                                                                                    fecha_staging = row.get('Material staging')
                                                                                                                                                                                                                                                                                                                                                                                                                                            try:
                                                                                                                                                                                                                                                                                                                                                                                                                                                        if pd.isnull(fecha_staging):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                        return None
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    return pd.to_datetime(str(fecha_staging), dayfirst=False).strftime('%Y-%m-%d')
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            except Exception as e:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        print(f"⚠️ Error al convertir 'Material staging' {fecha_staging}: {e}")
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    return None

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    # ------------------------------------------------------------------------- #
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    # Función: cargar la base consolidada anterior (si existe)                  #
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    # ------------------------------------------------------------------------- #
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    def cargar_base_anterior(output_folder):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        archivos = [f for f in os.listdir(output_folder) if f.startswith("consolidado_") and f.endswith(".xlsx")]
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            if not archivos:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    return pd.DataFrame()
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        archivo_mas_reciente = sorted(archivos)[-1]
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            ruta = os.path.join(output_folder, archivo_mas_reciente)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                return pd.read_excel(ruta, engine='openpyxl')

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # ------------------------------------------------------------------- #
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # Función: actualizar cambios MAD respecto a la base anterior         #
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # ------------------------------------------------------------------- #
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                def actualizar_cambios_mad(df_nuevo, df_anterior):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    df_merge = pd.merge(
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            df_nuevo,
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    df_anterior[['Key', 'Ultima_Fecha_MAD', 'Cambios_MAD']],
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            on='Key',
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    how='left',
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            suffixes=('', '_ant')
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                )
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    def calcular_cambios(row):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            if pd.isnull(row['Ultima_Fecha_MAD_ant']):
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        return 0
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                return row['Cambios_MAD_ant'] + 1 if row['Ultima_Fecha_MAD'] != row['Ultima_Fecha_MAD_ant'] else row['Cambios_MAD_ant']
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    df_merge['Cambios_MAD'] = df_merge.apply(calcular_cambios, axis=1)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        # Orden de columnas útil
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            cols_finales = ['Key', 'Ultima_Fecha_MAD', 'Cambios_MAD', 'Sales order no.', 'Material number', 'Text sales order', 'Material staging']
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # Agrega columnas que falten
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    for col in cols_finales:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            if col not in df_merge.columns:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        df_merge[col] = None
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            return df_merge[cols_finales]

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            # --------------------------- #
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            # Función principal del flujo #
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            # --------------------------- #
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            def main():
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                os.makedirs(OUTPUT_FOLDER, exist_ok=True)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    # 1. Cargar Excel(s) nuevo(s)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        df = cargar_archivos_excel(INPUT_FOLDER)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            # 2. Validación de columnas
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                columnas_esperadas = ['Sales order no.', 'Material number', 'Text sales order', 'Material staging']
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    for col in columnas_esperadas:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            if col not in df.columns:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        raise ValueError(f"Falta la columna requerida: {col}")

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            # 3. Crear columna Key
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                df = crear_columna_key(df)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    # 4. Calcular la fecha MAD final para cada fila
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        df['Ultima_Fecha_MAD'] = df.apply(obtener_ultima_fecha_mad, axis=1)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            # 5. Cargar base anterior
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                df_anterior = cargar_base_anterior(OUTPUT_FOLDER)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    # 6. Actualizar cambios MAD si hay base anterior
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        if not df_anterior.empty:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                df = actualizar_cambios_mad(df, df_anterior)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    else:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            df['Cambios_MAD'] = 0

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # 7. Guardar resultado
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    nombre_archivo = f'{OUTPUT_FOLDER}consolidado_{datetime.today().strftime("%Y%m%d")}.xlsx'
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        df.to_excel(nombre_archivo, index=False)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            print(f"✅ Base consolidada actualizada: {nombre_archivo}")

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            # ------------------------ #
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            # Ejecutar script principal#
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            # ------------------------ #
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            if __name__ == '__main__':
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                main()
